In [1]:
import re
import multiprocessing
from tqdm import tqdm

import pandas as pd
from wordcloud import WordCloud, STOPWORDS
from scipy.stats import norm
from gensim.models import word2vec
# from kaggle.competitions import twosigmanews

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

import numpy as np

%matplotlib inline

In [2]:
cpu_count = 2*multiprocessing.cpu_count()-1
print('Number of CPUs: {}'.format(cpu_count))

Number of CPUs: 55


In [4]:
# news_df = pd.read_csv("news_train_df.csv")
news_df = pd.read_csv("news_test_pred_df.csv")

In [5]:
america_df = pd.read_csv("America.csv")

In [6]:
america_df["time"] = pd.to_datetime(america_df["time"])
news_df["time"] = pd.to_datetime(pd.to_datetime(news_df['time']).dt.date)

In [7]:
america_df.head()

,time,close,open,high,low,volume,percent,code
0,2015-01-05,106.25,108.29,108.65,105.41,64.29M,-2.82%,AAPL
1,2015-01-06,106.26,106.54,107.43,104.63,65.80M,0.01%,AAPL
2,2015-01-07,107.75,107.20,108.20,106.69,40.11M,1.40%,AAPL
3,2015-01-08,111.89,109.23,112.15,108.70,59.36M,3.84%,AAPL
4,2015-01-09,112.01,112.67,113.25,110.21,53.70M,0.11%,AAPL


In [8]:
america_df['code'].unique()

array(['AAPL', 'V', 'BABA', 'ADBE', 'SNE'], dtype=object)

In [9]:
news_df = news_df[['assetCodes', 
                    'assetName', 
                    'sentimentClass', 
                    'time',
                    'sentimentNegative',
                    'sentimentNeutral',
                    'sentimentPositive']]

In [10]:
news_df['sentimentRange'] = np.where(
    news_df['sentimentNeutral'] > abs(
        news_df['sentimentPositive']-news_df['sentimentNegative']),
    0,
    news_df['sentimentPositive']-news_df['sentimentNegative']
)

In [11]:
news_test_df = news_df[ (news_df['time']>='2015-01-01')]
news_train_df = news_df[ (news_df['time']<'2015-01-01')]

In [12]:
# news_test_df['time'] = pd.to_datetime(news_test_df['time'].dt.date)

In [18]:
news_test_df['assetCodes'] = news_test_df['assetCodes'].map(flattenAssetCodes)

In [19]:
# news_test_df = news_df
news_test_df.head()

,Unnamed: 0,assetCodes,assetName,pred,headline,time,sentimentNegative,sentimentNeutral,sentimentPositive
0,7274526,{LONG},eLong Inc,-1,Beijing Court Rules in Favor of eLong in Dispu...,2015-01-01,0.809405,0.129914,0.060681
1,7274527,{EXPE},Expedia Inc,-1,Beijing Court Rules in Favor of eLong in Dispu...,2015-01-01,0.819121,0.125242,0.055638
2,7274528,{SYX},Systemax Inc,1,SYSTEMAX'S GLOBAL INDUSTRIAL BUSINESS SIGNS DE...,2015-01-01,0.034652,0.122020,0.843328
3,7274529,{SYX},Systemax Inc,1,Systemax's Global Industrial Business Signs De...,2015-01-01,0.097430,0.156117,0.746453
4,7274530,{BRO},Brown & Brown Inc,1,"Brown & Brown, Inc. Announces Sale of Certain ...",2015-01-01,0.552344,0.248000,0.199656


In [20]:
america_df["sentimentRange"] = 0.0

In [21]:
for index1, row1 in america_df.iterrows():
    tmp = []
    for index2, row2 in news_test_df[(news_test_df['time']==row1['time'])].iterrows():
        if row1['code'] in row2['assetCodes']:
#             if np.isnan(row1['sentimentRange']):
#                 row1['sentimentRange'] = 0
            tmp.append(row2['pred'])
    if len(tmp) > 0:
#         print(np.mean(tmp))  
        america_df.at[index1,'sentimentRange'] = np.mean(tmp)
#     break
    if index1 > 0 and index1 % 100 == 0:
        print("%d/%d"%(index1, len(america_df)))

100/3772
200/3772
300/3772
400/3772
500/3772
600/3772
700/3772
800/3772
900/3772
1000/3772
1100/3772
1200/3772
1300/3772
1400/3772
1500/3772
1600/3772
1700/3772
1800/3772
1900/3772
2000/3772
2100/3772
2200/3772
2300/3772
2400/3772
2500/3772
2600/3772
2700/3772
2800/3772
2900/3772
3000/3772
3100/3772
3200/3772
3300/3772
3400/3772
3500/3772
3600/3772
3700/3772


In [ ]:
# for index1, row1 in america_df.iterrows():
#     tmp = []
#     for index2, row2 in news_test_df[(news_test_df['time']==row1['time'])].iterrows():
#         if row1['code'] in row2['assetCodes']:
# #             if np.isnan(row1['sentimentRange']):
# #                 row1['sentimentRange'] = 0
#             tmp.append(row2['sentimentRange'])
#     if len(tmp) > 0:
# #         print(np.mean(tmp))  
#         america_df.at[index1,'sentimentRange'] = np.mean(tmp)
# #     break
#     if index1 > 0 and index1 % 100 == 0:
#         print("%d/%d"%(index1, len(america_df)))

In [44]:
# america_df = america_df.fillna(0)

In [22]:
america_df.head()

,time,close,open,high,low,volume,percent,code,sentimentRange
0,2015-01-05,106.25,108.29,108.65,105.41,64.29M,-2.82%,AAPL,0.200000
1,2015-01-06,106.26,106.54,107.43,104.63,65.80M,0.01%,AAPL,0.428571
2,2015-01-07,107.75,107.20,108.20,106.69,40.11M,1.40%,AAPL,-0.818182
3,2015-01-08,111.89,109.23,112.15,108.70,59.36M,3.84%,AAPL,-0.307692
4,2015-01-09,112.01,112.67,113.25,110.21,53.70M,0.11%,AAPL,-1.000000


In [23]:
america_df.to_csv("AmericaWithNewsPred.csv")

In [17]:
import ast
def flattenAssetCodes(codes):
    lst = ast.literal_eval(codes)
    def get1st(code):
        return code.split('.')[0]
    return set(map(get1st, lst))

In [70]:
strtst = "{'LONG.L', 'LONG.O', 'AAPL', 'KK.O'}"
flattenAssetCodes(strtst)

{'AAPL', 'KK', 'LONG'}

In [63]:
min(america_df['sentimentRange'])

-0.7635145000000001